<a href="https://colab.research.google.com/github/pranavkantgaur/training_materials/blob/master/ACT_office_demos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Common Imports
import numpy as np
import matplotlib.pyplot as plt
import time
import random

##Lecture 1: Naive Monte Carlo Simulation

Objective: Implement a serial neutron absorption simulator.

In [3]:
def simulate_serial(N=10000, p_abs=0.01, max_steps=100):
    absorbed = 0
    for _ in range(N):
        for _ in range(max_steps):
            if random.random() < p_abs:
                absorbed += 1
                break
    return absorbed

# Runtime and accuracy test
start = time.time()
result = simulate_serial(10000)  # Smaller N for quick demo
print(f"Absorbed: {result} | Time: {time.time() - start:.2f}s")

Absorbed: 6341 | Time: 0.17s


Explanation:

    Each neutron is simulated independently with a loop-in-loop structure.

    p_abs=0.01 means a 1% chance of absorption per step.



## C++ version

In [4]:
%%shell
cat << 'EOF' > neutron.cpp
#include <iostream>
#include <random>
#include <chrono>

int simulate_cpp(int N=10000, double p_abs=0.01, int max_steps=100) {
    std::random_device rd;
    std::mt19937 gen(rd());
    std::uniform_real_distribution<> dis(0.0, 1.0);

    int absorbed = 0;
    for (int i = 0; i < N; ++i) {
        for (int step = 0; step < max_steps; ++step) {
            if (dis(gen) < p_abs) {
                absorbed++;
                break;
            }
        }
    }
    return absorbed;
}

int main() {
    auto start = std::chrono::high_resolution_clock::now();
    int result = simulate_cpp();
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed = end - start;
    std::cout << "C++ Result: " << result
              << " | Time: " << elapsed.count() << "s\n";
    return 0;
}
EOF

In [5]:
%%shell
# Compile C++ code with optimizations
g++ -O3 -o neutron_simulation neutron.cpp
./neutron_simulation

C++ Result: 6362 | Time: 0.0132403s


## Time profile serial implementation

In [6]:
import cProfile

def simulate_serial(N=10000, p_abs=0.01, max_steps=100):
    absorbed = 0
    for _ in range(N):
        for _ in range(max_steps):
            if random.random() < p_abs:
                absorbed += 1
                break
    return absorbed

# Profile function
cProfile.run('simulate_serial(10000)', sort='cumulative')

         635155 function calls in 0.178 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.178    0.178 {built-in method builtins.exec}
        1    0.000    0.000    0.178    0.178 <string>:1(<module>)
        1    0.133    0.133    0.178    0.178 <ipython-input-6-1fb315fcf3de>:3(simulate_serial)
   635151    0.045    0.000    0.045    0.000 {method 'random' of '_random.Random' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




### Line by line profiling

In [7]:
!pip install line_profiler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.2/750.2 kB 12.3 MB/s eta 0:00:00


In [8]:
# Add this to your notebook
%load_ext line_profiler

def simulate_serial(N=10_000, p_abs=0.01, max_steps=100):
    absorbed = 0
    for _ in range(N):
        for _ in range(max_steps):
            if random.random() < p_abs:
                absorbed += 1
                break
    return absorbed

# Explicitly add the decorator (Colab workaround)
from line_profiler import LineProfiler
lp = LineProfiler()
lp_wrapper = lp(simulate_serial)

In [9]:
# Execute with profiling
result = lp_wrapper()
lp.print_stats()

Timer unit: 1e-09 s

Total time: 0.515393 s
File: <ipython-input-8-46150dfae621>
Function: simulate_serial at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def simulate_serial(N=10_000, p_abs=0.01, max_steps=100):
     5         1        988.0    988.0      0.0      absorbed = 0
     6     10001    3178235.0    317.8      0.6      for _ in range(N):
     7    637560  186472329.0    292.5     36.2          for _ in range(max_steps):
     8    633922  321429461.0    507.0     62.4              if random.random() < p_abs:
     9      6362    2384516.0    374.8      0.5                  absorbed += 1
    10      6362    1926060.0    302.7      0.4                  break
    11         1       1325.0   1325.0      0.0      return absorbed



Assignment:
1. Read specification of `random` module -- which probabolity distributions are supported? Also, comment on which one to use when.
2. What are range() objects in python?
3. How to improve the runtime performance without resorting to parallelization/vectorization?
   1. What is the penalty of `if` statement on overall performance? How to prevent that?
4. What does the `pip` command do? List steps. What are its alternatives?
5. How `cProfile.run('simulate_serial(10000)', sort='cumulative')` works?
6. Explain the working of:


```
from line_profiler import LineProfiler
lp = LineProfiler()
lp_wrapper = lp(simulate_serial)
result = lp_wrapper()
lp.print_stats()
```




## Lecture 2: Vectorized Implementation

Objective: Optimize using array operations.

# READ:https://chat.deepseek.com/a/chat/s/5ab04fdf-3e96-4ed6-87c2-513d4a518ae4

### With Numpy

In [ ]:
import numpy as np
from line_profiler import LineProfiler
from memory_profiler import profile

# Vectorized simulation
def simulate_numpy(N=10_000, p_abs=0.01, max_steps=100):
    # Generate all random numbers at once
    steps = np.random.rand(N, max_steps)  # Shape: (N, max_steps)

    # Check absorption per neutron (any step < p_abs)
    absorbed = np.any(steps < p_abs, axis=1).sum()

    return absorbed

# Line-level runtime profiling
lp = LineProfiler()
lp.add_function(simulate_numpy) # Add the function to be profiled
lp_wrapper = lp(simulate_numpy) # Wrap the function using lp

# Execute with profiling, this will call the wrapped function
result = lp_wrapper(N=10_000, p_abs=0.01, max_steps=100)

# Print line profiling stats
lp.print_stats()

# Print memory profiling results (will be printed to the console)

Timer unit: 1e-09 s

Total time: 0.0124622 s
File: <ipython-input-23-639abaf06017>
Function: simulate_numpy at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def simulate_numpy(N=10_000, p_abs=0.01, max_steps=100):
     7                                               # Generate all random numbers at once
     8         1   10791134.0    1e+07     86.6      steps = np.random.rand(N, max_steps)  # Shape: (N, max_steps)
     9                                               
    10                                               # Check absorption per neutron (any step < p_abs)
    11         1    1670348.0    2e+06     13.4      absorbed = np.any(steps < p_abs, axis=1).sum()
    12                                               
    13         1        748.0    748.0      0.0      return absorbed



In [ ]:
%%writefile my_script.py
import numpy as np

def simulate_numpy(N=10_000, p_abs=0.01, max_steps=100):
    # Generate all random numbers at once
    steps = np.random.rand(N, max_steps)  # Shape: (N, max_steps)

    # Check absorption per neutron (any step < p_abs)
    absorbed = np.any(steps < p_abs, axis=1).sum()

    return absorbed

Writing my_script.py


## C++ version

In [ ]:
%%shell
cat << 'EOF' > neutron_vectorized.cpp
#include <iostream>
#include <vector>
#include <random>
#include <chrono>
#include <algorithm>

int simulate_cpp_vectorized(int N=10000, double p_abs=0.01, int max_steps=100) {
    std::random_device rd;
    std::mt19937 gen(rd());
    std::uniform_real_distribution<> dis(0.0, 1.0);

    std::vector<bool> absorbed(N, false);  // Vectorized absorption flags

    for (int i = 0; i < N; ++i) {
        for (int step = 0; step < max_steps; ++step) {
            if (dis(gen) < p_abs) {
                absorbed[i] = true;
                break;
            }
        }
    }

    return std::count(absorbed.begin(), absorbed.end(), true);
}

int main() {
    auto start = std::chrono::high_resolution_clock::now();
    int result = simulate_cpp_vectorized();
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed = end - start;

    std::cout << "C++ Vectorized Result: " << result
              << " | Time: " << elapsed.count() << "s\n";
    return 0;
}
EOF

g++ -O3 -o neutron_vectorized neutron_vectorized.cpp && ./neutron_vectorized

C++ Vectorized Result: 6274 | Time: 0.0123285s


## Memory profiling

In [ ]:
from my_script import simulate_numpy
from memory_profiler import profile

# Memory profiling (run separately)
%mprun -f simulate_numpy simulate_numpy(100000)

Assignment:

1. How to reduce the memory usage of the vectorized version?
2. Why `absorbed` did not take memory?

## Lecture 3: Multiprocessing Parallelization

Objective: Distribute work across CPU cores.

In [ ]:
from multiprocessing import Pool

def simulate_neutron(args):
    p_abs, max_steps = args
    for _ in range(max_steps):
        if random.random() < p_abs:
            return 1
    return 0

def simulate_parallel(N=10_000, p_abs=0.01, max_steps=100):
    with Pool(2) as pool:  # Use 4 cores
        results = pool.map(simulate_neutron, [(p_abs, max_steps)] * N)
    return sum(results)
start = time.time()
print("Parallel Result:", simulate_parallel())
print(f"Parallel Time: {time.time() - start:.4f}s")

Parallel Result: 6328
Parallel Time: 0.0978s


Explanation:

    Pool.map splits N neutrons across workers.

    Each neutron simulation is independent (embarrassingly parallel).

Assignment:

    Benchmark performance for Pool(2) vs. Pool(8) on Google Colab.

## Lecture 4: GPU Acceleration with CuPy

Objective: Offload computation to GPU.

In [ ]:
!apt-get update
!apt-get install -y --no-install-recommends cuda-drivers

# Check driver version to confirm update
!nvidia-smi

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,381 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,784 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-upda

In [ ]:
!pip install cupy-cuda11x  # Install CuPy for Colab GPU

import cupy as cp

def simulate_gpu(N=10_000, p_abs=0.01, max_steps=100):
    steps = cp.random.rand(N, max_steps)
    return (cp.any(steps < p_abs, axis=1)).sum()

start = time.time()
result = simulate_gpu().get()  # Move data from GPU to CPU
print(f"GPU Result: {result} | Time: {time.time() - start:.4f}s")

CUDARuntimeError: cudaErrorInsufficientDriver: CUDA driver version is insufficient for CUDA runtime version

Explanation:

    cp.random.rand generates random numbers on the GPU.

    Operations like cp.any execute in parallel on GPU threads.

Assignment:

    Test with max_steps=1000 and compare GPU/CPU runtimes.

Lecture 5: Variance Reduction Techniques

Objective: Implement Russian Roulette for faster convergence.

In [ ]:
def simulate_russian_roulette(N=10_000, p_abs=0.01, survival_prob=0.5):
    absorbed = 0
    for _ in range(N):
        weight = 1.0
        while True:
            if random.random() < p_abs:
                absorbed += weight
                break
            # Russian Roulette: Kill neutron with 50% probability
            if random.random() > survival_prob:
                break
            weight /= survival_prob  # Adjust weight
    return absorbed

print("With Variance Reduction:", simulate_russian_roulette())

Explanation:

    Low-weight neutrons are probabilistically terminated to save computation.

    survival_prob balances computation and statistical bias.

Assignment:

    Compare convergence rates with/without Russian Roulette.

Lecture 6: OpenMC Reactor Simulation

Objective: Simulate a 3D fuel rod using OpenMC.

In [ ]:
!pip install --pre openmc
!python -m openmc.install

import openmc

# Define materials
fuel = openmc.Material()
fuel.add_element('U', 1.0, enrichment=4.25)
fuel.set_density('g/cm3', 10.0)

# Define geometry
sphere = openmc.Sphere(r=100.0)
cell = openmc.Cell(fill=fuel, region=-sphere)
geometry = openmc.Geometry([cell])

# Settings
settings = openmc.Settings()
settings.particles = 1000
settings.batches = 10

# Run simulation
model = openmc.Model(geometry=geometry, materials=openmc.Materials([fuel]), settings=settings)
model.run()

Explanation:

    OpenMC uses real nuclear data libraries (e.g., ENDF/B-VIII).

    Tallies track absorption, fission, etc., in 3D geometry.

Assignment:

    Add a water moderator around the fuel and compare absorption rates.

Lecture 7: Error Analysis

Objective: Compute statistical uncertainty.

In [ ]:
def simulate_with_error(N=10_000, p_abs=0.01, n_batches=10):
    results = []
    for _ in range(n_batches):
        absorbed = simulate_numpy(N // n_batches, p_abs)
        results.append(absorbed)
    mean = np.mean(results)
    std = np.std(results) / np.sqrt(n_batches)
    return mean, std

mean, std = simulate_with_error()
print(f"Absorption: {mean:.1f} ± {2*std:.1f} (95% CI)")

Explanation:

    Batches reduce correlation between samples.

    Standard error decreases as 1/sqrt(n_batches).

Assignment:

    Plot confidence intervals vs. number of batches.

Lecture 8: MPI for HPC

Objective: Scale simulations across nodes.

In [ ]:
!pip install mpi4py

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

def simulate_mpi(N=10_000, p_abs=0.01):
    chunk = N // size
    local_absorbed = simulate_numpy(chunk, p_abs)
    total = comm.reduce(local_absorbed, op=MPI.SUM)
    if rank == 0:
        return total

print("MPI Result:", simulate_mpi())

Explanation:

    mpi4py splits N neutrons across MPI ranks.

    comm.reduce aggregates results to rank 0.

Assignment:

    Run on 4 MPI processes and measure weak scaling efficiency.

Lecture 9: ML for Variance Reduction

Objective: Use a neural network to guide neutron paths.

In [ ]:
import tensorflow as tf

# Train a surrogate model to predict absorption probability
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='mse')

# Hybrid simulation (pseudo-code)
def simulate_ml(N=1000):
    absorbed = 0
    for _ in range(N):
        position = np.random.rand(3)  # 3D position
        p_abs_pred = model.predict(position.reshape(1, -1))[0][0]
        if random.random() < p_abs_pred:
            absorbed += 1
    return absorbed

Explanation:

    The neural network predicts location-dependent absorption probabilities.

    Simulations focus on high-probability regions.

Assignment:

    Train the model on OpenMC data and compare convergence rates.

Lecture 10: Final Project

Objective: Optimize a 2D reactor simulation.
Guidelines:

    Combine GPU acceleration (CuPy), variance reduction, and MPI.

    Compare runtime/accuracy trade-offs.

    Visualize neutron flux distribution.

In [ ]:
# 2D reactor core with materials
def simulate_2d_reactor(size=100):
    flux = np.zeros((size, size))
    for _ in range(N):
        x, y = np.random.randint(0, size, 2)
        # Track neutron path in 2D grid
        ...
    return flux